In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **EDA**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import missingno as mno
import plotly.offline as pyo 
import plotly.figure_factory as ff
import plotly.io as pio
from wordcloud import WordCloud
color_pal = sns.color_palette()
plt.style.use('seaborn-dark-palette')
plt.style.use('dark_background')

import nltk
import re

import warnings
warnings.filterwarnings('ignore')
sns.set_theme(style='darkgrid', palette='colorblind')
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()

#Model
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
df=pd.read_csv('/kaggle/input/hotel-price-data-of-cities-in-india-makemytrip/bangalore.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
df.describe().T

In [ ]:
df.describe(include='object').T

# **NULL AND MAKE THIS RESULT MAGIC**

In [ ]:
print(df.isna().sum())
df.isna().sum().plot(kind='bar')
plt.show()
sns.barplot(df.isna())
plt.xticks(rotation=45)
plt.show()

In [ ]:
#heatmap
plt.figure(figsize = (10,4))
sns.heatmap(df.isna().transpose(),
            cmap = "YlGnBu",
            cbar_kws = {'label': 'Missing Data'})
plt.show()

# **Convert And Drop ","**

In [ ]:
# Remove commas from 'Price' column and convert to float
df['Price'] = df['Price'].str.replace(',', '').astype(float)

# Remove commas from 'Tax' column and convert to float
df['Tax'] = df['Tax'].str.replace(',', '').astype(float)


# **FILL AND DROP NULL**

In [ ]:
# Fill missing values in the 'Rating' column with the mode
df['Rating'].fillna(df['Rating'].mode()[0], inplace=True)

# Fill missing values in the 'Rating Description' column with 'Unknown'
df['Rating Description'].fillna('Unknown', inplace=True)

# Fill missing values in the 'Reviews' column with the mode
df['Reviews'].fillna(df['Reviews'].mode()[0], inplace=True)

# Fill missing values in the 'Star Rating' column with the median
median_Star_Rating = df['Star Rating'].median()
df['Star Rating'].fillna(median_Star_Rating, inplace=True)

# Fill missing values in the 'Tax' column with the mean
mean_Tax = df['Tax'].mean()
df['Tax'].fillna(mean_Tax, inplace=True)

# Drop the columns from the DataFrame
df.drop(columns=['Nearest Landmark', 'Distance to Landmark'], inplace=True)

# Clean the 'Hotel Name' values by removing commas, forward slashes, hyphens, periods, and backslashes
df['Hotel Name'] = df['Hotel Name'].apply(lambda x: re.sub(r'[,/.\\-]', '', x))

# Display the shape of the DataFrame after changes
print(df.shape)

In [ ]:
df.head()

# **duplicated**

In [ ]:
df.duplicated().sum()

# **RENAME**

In [ ]:
# Renaming the Columns
df.rename(columns ={'Rating Description':'Rating_Description',
                    'Star Rating':'Star_Rating',
                    'Nearest Landmark':'Nearest_Landmark',
                    'Distance to Landmark': 'Distance_to_Landmark'}, inplace = True)
# Add 'Price' and 'Tax' columns element-wise and update 'Price' column
df['Price'] = df['Price'] + df['Tax']

# Drop the columns from the DataFrame
df.drop(columns=['Tax'], inplace=True)
df.head()

# **Visualizing**

In [ ]:
# Import the ProfileReport class from the ydata_profiling library
from ydata_profiling import ProfileReport

# Create a comprehensive profile report for the DataFrame 'df'
# This report will contain various statistics, insights, and visualizations about the data
profile = ProfileReport(df)

In [ ]:
profile                     # or this one
profile.to_notebook_iframe() # use this line to show the output

In [ ]:
locationLocation_of_Studnet = df.groupby('Location')['Price'].sum().reset_index()

# Create a line plot using Plotly Express
# x-axis: locationLocation, y-axis: Location ,
# labels for the x-axis, title, and customized height
locationLocation_of_Studnet = px.line(
    locationLocation_of_Studnet,  # DataFrame containing the data
    x='Location',   # x-values
    y='Price',  # y-values
    title='Price  by location',  # Set the title of the plot
    height=800  # Set the height of the plot
)

# Display the plot
locationLocation_of_Studnet.show()

In [ ]:
# Correlation:
plt.figure(figsize=(10, 6))
sns.heatmap(df.drop(['Rating_Description','Location',], axis = 1).corr(), cmap="Blues", annot = True)

In [ ]:
plt.figure(figsize=(10, 4))
plt.bar(df.Location, df.Price)
plt.xticks(rotation=90)
plt.show()

plt.figure(figsize=(10, 4))
plt.bar(df.Location, df.Reviews)
plt.xticks(rotation=90)
plt.show()

In [ ]:
df.head()

In [ ]:
df['Rating_Description'].value_counts().plot(kind = 'barh', figsize = (10,2), grid = True, color='black')
plt.title('Rating_Description Amount')
plt.xlabel('Amount')
plt.ylabel('Rating_Description')
plt.show()

In [ ]:
# Calculate the value counts for each unique value in the 'job_title' column
top10_Location = df['Location'].value_counts()[:10]

# Create a bar plot using Plotly Express
# y-axis: Count of each job title, x-axis: Location,
# color-coded by job titles and using a color palette
fig = px.bar(
    y=top10_Location.values,  # y-values: count of each job title
    x=top10_Location.index,   # x-values: job titles
    color=top10_Location.index,  # Color the bars based on job titles
    color_discrete_sequence=px.colors.sequential.PuBuGn,  # Set color palette
    text=top10_Location.values,  # Display the count values on top of the bars
    title='Top 10 Location',  # Set the title of the plot
    template='plotly_white'  # Use a dark template for the plot
)

# Update the layout of the plot
fig.update_layout(
    title_text='Location Distribution',  # Set the title of the plot
    height=700,  # Set the height of the plot
    xaxis_title="Location",  # Label for the x-axis
    yaxis_title="Count",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
# Calculate the average salary for each Hotel Name and sort in descending order
average_Price = df.groupby('Hotel Name')['Price'].mean().reset_index()
average_Price = average_Price.sort_values(by='Price', ascending=False)

# Select the top 10 Hotel Name with the highest average price
top_10_Price = average_Price.head(10)

# Create a new figure using Plotly
fig = go.Figure()

# Add a bar trace to the figure
fig.add_trace(go.Bar(
    x=top_10_Price['Hotel Name'],  # x-values: Hotel Name
    y=top_10_Price['Price']  # y-values: average Price
))

# Update the layout and appearance of the plot
fig.update_layout(
    title='Top 10 Hotel by Price',  # Set the title of the plot
    xaxis_title='Hotel Name',  # Label for the x-axis
    yaxis_title='Price',  # Label for the y-axis
    #paper_bgcolor='#595964',  # Background color of the plot
    template='plotly_white',  # Use a dark template for the plot
    font=dict(color='black'),  # Set font color to white
    height=800  # Set the height of the plot
)

# Initialize Plotly for notebook integration
pyo.init_notebook_mode(connected=True)

# Display the plot
pyo.iplot(fig)

In [ ]:
# Calculate the average salary for each Hotel Name and sort in descending order
average_Reviews = df.groupby('Hotel Name')['Reviews'].mean().reset_index()
average_Reviews = average_Reviews.sort_values(by='Reviews', ascending=False)

# Select the top 10 Hotel Name with the highest average price
top_10_Reviews = average_Reviews.head(10)

# Create a new figure using Plotly
fig = go.Figure()

# Add a bar trace to the figure
fig.add_trace(go.Bar(
    x=top_10_Reviews['Hotel Name'],  # x-values: Hotel Name
    y=top_10_Reviews['Reviews']  # y-values: Reviews Price
))

# Update the layout and appearance of the plot
fig.update_layout(
    title='Top 10 Hotel by Reviews',  # Set the title of the plot
    xaxis_title='Hotel Name',  # Label for the x-axis
    yaxis_title='Reviews',  # Label for the y-axis
    #paper_bgcolor='#595964',  # Background color of the plot
    template='plotly_white',  # Use a dark template for the plot
    font=dict(color='black'),  # Set font color to white
    height=900  # Set the height of the plot
)

# Initialize Plotly for notebook integration
pyo.init_notebook_mode(connected=True)

# Display the plot
pyo.iplot(fig)

In [ ]:
 # x-axis: "locationLocation", y-axis: "Number of student per staffs", color-coded by "Number of Studnet"
fig = px.bar(df, x="Rating_Description", y="Reviews", color="Reviews")

# Update the layout of the plot
fig.update_layout(
    title_text='Rating_Description Distribution with Number of Reviews',  # Set the title of the plot
    height=650,  # Set the height of the plot
    xaxis_title="Rating_Description",  # Label for the x-axis
    yaxis_title="Reviews",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
# Sunburst chart for the distribution of Rating_Description
fig2 = px.sunburst(df, path=['Rating_Description'], color_discrete_sequence=px.colors.qualitative.Set2)
fig2.update_layout(title_text='Distribution of Rating_Description', height=500)
fig2.show()

# Sunburst chart for the distribution of Rating
fig2 = px.sunburst(df, path=['Rating'], color_discrete_sequence=px.colors.qualitative.Set1)
fig2.update_layout(title_text='Distribution of Rating', height=500)
fig2.show()

In [ ]:
import plotly.express as px
fig = px.scatter(df, x="Rating_Description", y="Price", color="Rating")
fig.show()

fig = px.scatter(df, x="Price", y="Rating", size="Price",color="Rating_Description",
           hover_name="Location", log_x=True, size_max=60)
fig.show()

In [ ]:
df.head()


In [ ]:
fig = px.funnel(df, x='Star_Rating', y='Rating_Description')
fig.show()

In [ ]:
fig = px.box(df, x="Rating_Description", y="Price",color='Rating', notched=True)
fig.show()

In [ ]:
# Distribution of Popularity
#plt.figure(figsize=(10, 6))
sns.histplot(df['Rating_Description'], kde=True)
plt.title('Distribution of Rating_Description')
plt.xlabel('Rating_Description')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Box Plot
# Prepare data for the box plot
hist_data = [df['Price']]
group_labels = ['Price']

# Create the box plot using Plotly Express
fig1 = px.box(y=df['Price'], template='plotly_white', title='Price (BoxPlot)')

# Customize the layout of the box plot
fig1.update_layout(font=dict(size=17, family="Franklin Gothic"))

# Display the box plot
fig1.show()

# Dist Plot
# Prepare data for the distribution plot
hist_data = [df['Price']]
group_labels = ['Price']

# Create the distribution plot using Plotly Figure Factory
fig2 = ff.create_distplot(hist_data, group_labels, show_hist=False)
fig2.layout.template = 'plotly_white'

# Customize the layout of the distribution plot
fig2.update_layout(title='Price (DistPlot)', font=dict(size=17, family="Franklin Gothic"))

# Display the distribution plot
fig2.show()

In [ ]:
sns.pairplot(df)

In [ ]:
# Select columns with object (categorical) data types
num_cols = df.select_dtypes(include='object').columns.tolist()

# Initialize the LabelEncoder
le = LabelEncoder()

# Apply Label Encoding to the selected numerical columns
for x in num_cols:  
    df[x] = le.fit_transform(df[x])

# Now, your categorical columns (excluding column 0) have been converted to numerical values
df.head()

In [ ]:
Corr_Matrix = df.corr()

# Set up the figure and plot the heatmap
plt.figure(figsize=(10, 10))
sns.heatmap(Corr_Matrix, annot=True, cmap='RdYlGn', center=0)
plt.show()

# **Top 5 Most Positively**

In [ ]:
print('Top 5 Most Positively Correlated to the To salary_in_usd')
Corr_Matrix['Price'].sort_values(ascending=False).head(5)

# **Top 5 Most Positively**

In [ ]:
print('Top 5 Most Positively Correlated to salary_in_usd ')
Corr_Matrix['Price'].sort_values(ascending=True).head(5)

# **Split And Model**

In [ ]:
X = df.drop(columns=['Price','Rating_Description'])
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Display the shapes of the resulting datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
models = {
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
}
best_model = None
best_r2 = 0

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred= model.predict(X_test)

    # Evaluate the model
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    submit = pd.DataFrame()
    submit['Actual_salary'] = y_test
    submit['Predict_salary'] = y_pred
    submit = submit.reset_index()
    r2 = r2_score(y_test, y_pred)
    if r2 > best_r2:
        best_r2 = r2
        best_model = model.__class__.__name__

    print(f'{model_name}:')
    print(f'R2 Score: {r2:.2f}')
    print(f'Mean Absolute Error (MAE): {mae:.2f}')
    print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
    print(submit.head(5))

    print('----------------------------------------')
print(f"The best performing model is: {best_model} with accuracy: {best_r2:.2f}")

# **feature_importances**

In [ ]:
importances = model.feature_importances_

feature_names = X.columns

feature_importance_dict = dict(zip(feature_names, importances))

sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

for feature, importance in sorted_feature_importance:
    print(f"{feature}: {importance:.2f}")

plt.figure(figsize=(12, 7))
plt.barh(*zip(*sorted_feature_importance), alpha=0.9, color='red')
plt.title('Feature Importance', fontsize=15)
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()